<a href="https://colab.research.google.com/github/jmuddappa/cdqa_roberta/blob/master/roberta_cdqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 13505, done.
remote: Total 13505 (delta 0), reused 0 (delta 0), pack-reused 13505
Receiving objects: 100% (13505/13505), 6.28 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (9942/9942), done.


In [3]:
%cd fairseq
!pip install --editable .

/content/fairseq
Obtaining file:///content/fairseq
  Running setup.py develop for fairseq


In [4]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
!tar -xvf /content/fairseq/roberta.large.tar.gz
!bash examples/roberta/commonsense_qa/download_cqa_data.sh


--2020-02-14 22:06:36--  https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655283069 (625M) [application/gzip]
Saving to: ‘roberta.large.tar.gz’

roberta.large.tar.g 100%[===================>] 624.93M  36.2MB/s    in 18s     

2020-02-14 22:06:54 (35.2 MB/s) - ‘roberta.large.tar.gz’ saved [655283069/655283069]

roberta.large/
roberta.large/dict.txt
roberta.large/model.pt
roberta.large/NOTE
--2020-02-14 22:07:07--  https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.198
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785890 (3.6M) [binary/octet-stre

In [5]:
%env MAX_UPDATES=3000
%env WARMUP_UPDATES=150
%env LR=1e-05
%env MAX_SENTENCES=8
%env SEED=1
%env ROBERTA_PATH=/roberta.large/model.pt
%env DATA_DIR=/content/fairseq/data/CommonsenseQA

%env FAIRSEQ_PATH=/content/fairseq
%env FAIRSEQ_USER_DIR=/content/fairseq/examples/roberta/commonsense_qa

!CUDA_VISIBLE_DEVICES=0 fairseq-train --fp16 --ddp-backend=no_c10d $DATA_DIR --user-dir $FAIRSEQ_USER_DIR \
    --restore-file $ROBERTA_PATH \
    --reset-optimizer --reset-dataloader --reset-meters \
    --no-epoch-checkpoints --no-last-checkpoints --no-save-optimizer-state \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --task commonsense_qa --init-token 0 --bpe gpt2 \
    --arch roberta_large --max-positions 512 \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --criterion sentence_ranking --num-classes 5 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 1e-06 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $LR \
    --warmup-updates $WARMUP_UPDATES --total-num-update $MAX_UPDATES \
    --max-sentences $MAX_SENTENCES \
    --max-update $MAX_UPDATES \
    --log-format simple --log-interval 25 \
    --seed $SEED

env: MAX_UPDATES=3000
env: WARMUP_UPDATES=150
env: LR=1e-05
env: MAX_SENTENCES=8
env: SEED=1
env: ROBERTA_PATH=/roberta.large/model.pt
env: DATA_DIR=/content/fairseq/data/CommonsenseQA
env: FAIRSEQ_PATH=/content/fairseq
env: FAIRSEQ_USER_DIR=/content/fairseq/examples/roberta/commonsense_qa
2020-02-14 22:07:11 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='gelu', adam_betas='(0.9, 0.98)', adam_eps=1e-06, all_gather_list_size=16384, arch='roberta_large', attention_dropout=0.1, best_checkpoint_metric='accuracy', bpe='gpt2', broadcast_buffers=False, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='sentence_ranking', curriculum=0, data='/content/fairseq/data/CommonsenseQA', dataset_impl=None, ddp_backend='no_c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.1, empty_cache_freq=0, encoder_attention_